 1. Set Up Environment

In [2]:
# Create and activate virtual environment
# python3 -m venv env
# source env/bin/activate

# Install OpenNMT-py
!pip install OpenNMT-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Get and Prepare Parallel Data
  
  Use JW300 as an example.

In [ ]:
# Install opus-tools to download datasets
sudo apt install opus-tools

# Download JW300 English-Zulu parallel corpus
opus_get JW300 -s en -t zu -dl JW300_en_zu


Clean and format the data:

In [ ]:
# Assuming data is in JW300_en_zu/
# Create cleaned source and target files
cut -f1 JW300_en_zu/JW300.en-zu.en > train.en
cut -f2 JW300_en_zu/JW300.en-zu.zu > train.zu


You should also create a small validation set:

In [ ]:
head -n 500 train.en > valid.en
head -n 500 train.zu > valid.zu
tail -n +501 train.en > train_clean.en
tail -n +501 train.zu > train_clean.zu


Preprocess the Data

In [ ]:
onmt_preprocess \
  -train_src train_clean.en \
  -train_tgt train_clean.zu \
  -valid_src valid.en \
  -valid_tgt valid.zu \
  -save_data data/jw300 \
  -src_seq_length 100 \
  -tgt_seq_length 100 \
  -share_vocab \
  -src_vocab_size 16000 \
  -tgt_vocab_size 16000


 4. Train the Model

In [ ]:
onmt_train \
  -data data/jw300 \
  -save_model models/en2zu \
  -world_size 1 \
  -gpu_ranks 0 \
  -train_steps 10000 \
  -valid_steps 1000 \
  -batch_size 64


Translate with the Trained Model

In [ ]:
echo "This is a book." > src.txt

onmt_translate \
  -model models/en2zu_step_10000.pt \
  -src src.txt \
  -output pred.txt \
  -gpu 0


6. Evaluate

  Install BLEU scorer:

In [ ]:
pip install sacrebleu

# Reference: valid.zu
# Prediction: pred.txt
sacrebleu valid.zu -i pred.txt -m bleu
